In [ ]:
%pip install python-dotenv beautifulsoup4.10 selenium

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from dotenv import dotenv_values
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

In [ ]:
current_folder = globals()['_dh'][0]
filepath = os.path.join(current_folder,'chromedriver.exe')
filepath

In [ ]:
config = dotenv_values(".env")

In [ ]:
DOMAIN='http://www.facebook.com'
SEED_URL='https://m.facebook.com/ondemandacademy/?tsid=0.7139138908850204&source=result' 
scroll_pause_time = 1 # set your own pause time. 
break_year='2020'

Disable Alerts/Notifications
!! Using code by pythonjar of Stackoverflow !!
Please visit this thread for more details:
https://stackoverflow.com/questions/41400934/handle-notifications-in-python-selenium-chrome-webdriver

In [ ]:
#code by pythonjar
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

Log into personal Facebook account

In [ ]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome(filepath, chrome_options=chrome_options)

#open the webpage
driver.get(DOMAIN)

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys(config['USERNAME'])
password.clear()
password.send_keys(config['PASSWORD'])

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
time.sleep(2)  # Allow 2 seconds for the web success to login

#We are logged in!

In [ ]:
# Web scrapper for infinite scrolling page 
driver.get(SEED_URL)
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i=1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    time.sleep(scroll_pause_time)
    i+=1
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    soup = BeautifulSoup(driver.page_source,'html.parser')
    date_ele = soup.find_all('div',{'data-sigil':'m-feed-voice-subtitle'})[-1]
    print(f'page: {i}')
    print(date_ele.text)
    # Break the loop when post timestamp on break date.
    if  break_year in date_ele.text:
        break

In [ ]:
soup2 = BeautifulSoup(driver.page_source,'html.parser')
soup2.prettify()

In [ ]:
comments_ele = soup2.find_all('a',{'data-ft':'{"tn":"S"}'})
comments_ele

In [ ]:
post_link = [urljoin(SEED_URL,ele['href']) for ele in comments_ele]
post_link

In [ ]:
f = open("post_link.txt", "w")
f.write("\n".join(post_link))
f.close()

In [ ]:
import codecs

In [ ]:
all_posts = []
for link in post_link:
    post = []
    driver.get(link)
    time.sleep(1)
    soup3 = BeautifulSoup(driver.page_source, "html.parser")
    date = soup3.find("abbr")
    post.append(date.text)
    content = soup3.find("div",{'data-ft':'{"tn":"*s"}'})
    if(content):
        post.append(content.text.replace('\u200d♂️See translation',''))
    else: post.append(None)
    react = soup3.find('div',{'class':"_1g06"})
    if(react):
        post.append(react.text)
    else: post.append(None)
    shares = soup3.find('span',{'data-sigil':"feed-ufi-sharers"})
    if(shares):
        post.append(shares.text.replace(' shares','').replace(' share',''))
    else: post.append(None)
    comments = soup3.find_all('div',{'class':"_2b06"})
    if(comments):
        post_comment = []
        for comment_box in comments:
            post_comment.append([e.text for e in comment_box.findChildren(recursive=False)])
        post.append(post_comment)
    else: post.append(None)
    print(post)
    all_posts.append(post)
all_posts


In [ ]:
%pip install numpy



In [ ]:
%pip install pandas

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(all_posts,columns=["Date","content","likes","shares","comment"])
print(df)

In [ ]:
df.to_csv('sample.csv',encoding='utf-8')